In [375]:
import pandas as pd

In [487]:
train_set = pd.read_csv('relevance_train.csv', sep='\t', names=['query_id', 'document_id', 'relevance'])
train_set = train_set[1:]
train_set.head()

,query_id,document_id,relevance
1,1,184,2
2,1,29,2
3,1,31,2
4,1,12,3
5,1,51,3


In [488]:
test_set = pd.read_csv('relevance_test.csv', sep=',', names=['query_id', 'document_id'])
test_set = test_set[1:]
test_set.head()

,query_id,document_id
1,126,974
2,126,1326
3,126,187
4,126,969
5,126,970


In [489]:
queries_set = pd.read_csv('queries.csv', sep='\t', names=['query_id', 'query'], header=None, index_col=['query_id'])
queries_set = queries_set[1:]
queries_set.head()

,query
query_id,
1,what similarity laws must be obeyed when const...
2,what are the structural and aeroelastic proble...
4,what problems of heat conduction in composite ...
8,can a criterion be developed to show empirical...
9,what chemical kinetic system is applicable to ...


In [490]:
# documents_set = pd.DataFrame(columns=['.Id', '.A', '.T', '.B','.W'])
documents_set = pd.DataFrame(columns=['document_id', 'content'])
id_pattern = '.Id'
author_pattern = '.A'
theme_pattern = '.T'
date_pattern = '.B'
cont_pattern = '.W'
patterns = ['.Id', '.A', '.T', '.B','.W']

In [491]:
# documents_set.add(0)
documents_set.head()

,document_id,content


In [492]:
with open('Documents.csv') as data_file:
    doc_content = ""
    for line in data_file:
        is_pattern = False
        if (line.startswith(id_pattern)):
            if (doc_content != ""):
                documents_set = documents_set.append({'document_id':doc_id, 'content':Counter(doc_content.split(' '))}, ignore_index=True)
            doc_content = ""  
            doc_id = int(line[3:])
            continue

        for pattern in patterns:
            if (line.startswith(pattern)):
                is_pattern = True
                break
                
        if (not is_pattern):
            doc_content += " ".join(line.split('\n'))

In [493]:
documents_set = documents_set.set_index('document_id')
documents_set = pd.DataFrame(documents_set)
documents_set.head()

,content
document_id,
1,"{'experimental': 3, 'investigation': 2, 'of': ..."
2,"{'simple': 3, 'shear': 3, 'flow': 7, 'past': 5..."
3,"{'the': 3, 'boundary': 2, 'layer': 2, 'in': 2,..."
4,"{'approximate': 2, 'solutions': 3, 'of': 5, 't..."
5,"{'one-dimensional': 2, 'transient': 3, 'heat':..."


In [494]:
# documents_set.iloc[5, :][0]

In [495]:
from functools import reduce
import numpy as np

In [496]:
print(len(queries_set['query']))
# print("".join(list(filter(lambda x : type(x)==str, np.array(queries_set['query'])))))

226


In [497]:
unique_query_words = []
unique_query_words = "".join(list(filter(lambda x : type(x)==str, np.array(queries_set['query']))))
unique_query_words = np.unique(unique_query_words.split(" "))
# for query in queries_set['query']:
#     unique_query_words.append[]

In [498]:
meta_charact = ['.', '^', '$', '*', '+', '?', '{', '}', '[', ']', '\\', '|', '(' ,')']

In [499]:
# unique_query_words[:50]

In [500]:
print(train_set.iloc[1, :])

query_id        1
document_id    29
relevance       2
Name: 2, dtype: object


In [501]:
queries_set.head()
print(type(queries_set))

<class 'pandas.core.frame.DataFrame'>


In [502]:
train_set_res = train_set.join(queries_set, on='query_id', lsuffix="_train", rsuffix="_queries")
train_set_res.head()

,query_id,document_id,relevance,query
1,1,184,2,what similarity laws must be obeyed when const...
2,1,29,2,what similarity laws must be obeyed when const...
3,1,31,2,what similarity laws must be obeyed when const...
4,1,12,3,what similarity laws must be obeyed when const...
5,1,51,3,what similarity laws must be obeyed when const...


In [503]:
from collections import Counter

In [504]:
t = Counter('sdfsd sdfdf'.split(' '))
t['sds']
for i, s in enumerate([1, 2]):
    print(i)

0
1


In [505]:
s = 'dfd dsfsd, sdf'
# s.split(sep=[' ', ','])

In [506]:
with open('train.txt', 'w') as features_matrix_train:
    with open('validation.txt', 'w') as features_matrix_valid:
        for row_ind in range(len(train_set_res)):
            line = ""
            query_id = train_set_res.iloc[row_ind, :][0]
            document_id = train_set_res.iloc[row_ind, :][1]
            relevance = train_set_res.iloc[row_ind, :][2]
            query_content = train_set_res.iloc[row_ind, :][3]
            query_content = str(query_content).split(' ')
#             print(row_ind, relevance, query_id)
            document_content_counter = documents_set.iloc[int(document_id), :][0]

            line += str(relevance) + ' ' + 'qid:' + str(query_id) + ' '

            for ind, word in enumerate(unique_query_words):
                if (not word in query_content):
                    line += str(ind) + ':' + '0' + ' '
                    continue
                line += str(ind) + ':' + str(document_content_counter[word]) + ' '

            line += '\n'
            
            if (row_ind < round(len(train_set_res)*0.7)):
                features_matrix_train.write(line)
            else:
                features_matrix_valid.write(line)

In [507]:
test_set_res = test_set.join(queries_set, on='query_id', lsuffix="_train", rsuffix="_queries")
test_set_res.head()

,query_id,document_id,query
1,126,974,what are wind-tunnel corrections for a two-dim...
2,126,1326,what are wind-tunnel corrections for a two-dim...
3,126,187,what are wind-tunnel corrections for a two-dim...
4,126,969,what are wind-tunnel corrections for a two-dim...
5,126,970,what are wind-tunnel corrections for a two-dim...


In [508]:
test_set_res = test_set.join(queries_set, on='query_id', lsuffix="_train", rsuffix="_queries")
test_set_res.head(4)

,query_id,document_id,query
1,126,974,what are wind-tunnel corrections for a two-dim...
2,126,1326,what are wind-tunnel corrections for a two-dim...
3,126,187,what are wind-tunnel corrections for a two-dim...
4,126,969,what are wind-tunnel corrections for a two-dim...


In [509]:
test_set_res.iloc[0, :][1]

'974'

In [510]:
with open('test.txt', 'w') as features_matrix:
    i = 0
    for row_ind in range(len(test_set_res)):
        line = ""
        query_id = test_set_res.iloc[row_ind, :][0]
        document_id = test_set_res.iloc[row_ind, :][1]
        query_content = test_set_res.iloc[row_ind, :][2]
        query_content = str(query_content).split(' ')
        line += '-1' + ' ' + 'qid:'+str(query_id) + ' '
        i += 1
    
        try:
            document_content_counter = documents_set.iloc[int(document_id), :][0]
        except IndexError:
            print(i, query_id, document_id)
            for ind in range(len(unique_query_words)):
                line += str(ind) + ':' + '0' + ' '
            line += '\n'
            features_matrix.write(line)
            continue

        for ind, word in enumerate(unique_query_words):
            if (not word in query_content):
                line += str(ind) + ':' + '0' + ' '
                continue
            line += str(ind) + ':' + str(document_content_counter[word]) + ' '

        line += '\n'
        features_matrix.write(line)

804 222 1398
805 222 1397
806 222 1400
807 222 1399
812 223 1398


In [511]:
%%time
! java -jar RankLib-2.1-patched.jar -train train.txt -rank test.txt -validate validation.txt -ranker 6 -tree 300 -metric2t NDCG@5 -save log_LambdaMART > log_LambdaMART.txt

Wall time: 7.04 s


In [512]:
!java -jar RankLib-2.1-patched.jar -load log_LambdaMART -rank test.txt -score myscorefile.txt


[+] General Parameters:
Model file:	log_LambdaMART
Feature normalization: No
Model:		LambdaMART

Reading feature file [test.txt]: 0... 
Reading feature file [test.txt]... [Done.]            
(100 ranked lists, 847 entries read)


In [542]:
score = pd.read_csv('myscorefile.txt', sep='\t', header=None)
score = score.drop(1, axis=1)
score.head()

,0,2
0,126,0.464898
1,126,-0.038361
2,126,0.464898
3,126,-0.038361
4,126,0.464898


In [543]:
test_rel = pd.read_csv('relevance_test.csv', sep=',', names=['query_id', 'document_id'])
test_rel = test_rel[1:]
test_rel.head()
len(test_rel) == len(score)

True

In [544]:
test_rel.index = range(len(test_rel))
test_rel['document_id']
test_rel.head()

,query_id,document_id
0,126,974
1,126,1326
2,126,187
3,126,969
4,126,970


In [545]:
score['3'] = test_rel['document_id'][0:]
score.head()

,0,2,3
0,126,0.464898,974
1,126,-0.038361,1326
2,126,0.464898,187
3,126,-0.038361,969
4,126,0.464898,970


In [546]:
output = score.groupby(0).apply(lambda x: x.sort_values([2],ascending=False).head(len(score)))
output.index = range(len(output))
output = pd.DataFrame(output)
output = output.drop(2, axis=1)
output.columns = ['QueryId','DocumentId']
output.head(9)
# 126,974
# 126,1326
# 126,187
# 126,969
# 126,970
# 126,971
# 126,972
# 126,973
# 126,942


,QueryId,DocumentId
0,126,974
1,126,187
2,126,970
3,126,942
4,126,971
5,126,1326
6,126,969
7,126,973
8,126,972


In [548]:
output.to_csv('out.csv', index=None)